# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [36]:
G1 = nx.read_gpickle("assets/A4_P1_G1")
G2 = nx.read_gpickle("assets/A4_P1_G2")
G3 = nx.read_gpickle("assets/A4_P1_G3")
G4 = nx.read_gpickle("assets/A4_P1_G4")
G5 = nx.read_gpickle("assets/A4_P1_G5")
P1_Graphs = [G1, G2, G3, G4, G5]

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs using any methodology and determine which of the three algorithms generated each graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [37]:
def graph_identification():
    
    import matplotlib.pyplot as plt
    import pandas as pd
    # Calculate average clustering, average shortedt distance
    # Plot average clustering v. average distance on plot
    # can use that to distinguish

#     data = {}
#     i = 0
#     for graph in P1_Graphs:
#         name = f'Graph {i}'
#         data[name] = {'List Index': P1_Graphs.index(graph),
#                       'Num Nodes': len(graph.nodes),
#                       'Num Edges': len(graph.edges),
#                       'Average Clustering': nx.average_clustering(graph),
#                       'Average Shortest Path': nx.average_shortest_path_length(graph)}
#         i += 1
    
#     data = pd.DataFrame(data).T

# #     avc = [data[graph]['Average Clustering'] for graph in data.keys()]
# #     avsp = [data[graph]['Average Shortest Path'] for graph in data.keys()]
    
    
#     plt.figure(figsize=(5,5))
#     plt.plot(data['Average Clustering'], data['Average Shortest Path'], 'ro')
#     for label in list(data.index):
#         coordinates = tuple(data.loc[label])
#         plt.text(coordinates[3], coordinates[4], label)
    

    
    
    # raise NotImplementedError()
    # graph 0, 3 = Preferential Attachment
    #              Reasoning: same number of edges as vertices, no clustering because most nodes are created and attach to central node
    # graph 1, 2 = Small World with (large/small) p. 
    #              Reasoning: Large average path length means p is small (fewer longer connections made)
    # graph 4 = Small Workd with large p (elimination)
    
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

# graph_identification()

In [ ]:
ans_one = graph_identification()
assert type(ans_one) == list, "You must return a list"


---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [38]:
G = pickle.load(open('assets/email_prediction_NEW.txt', 'rb'))

print(f"Graph with {len(nx.nodes(G))} nodes and {len(nx.edges(G))} edges")

Graph with 1005 nodes and 16706 edges


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features of your choice using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a Pandas series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [39]:
list(G.nodes(data=True))[:5] # print the first 5 nodes

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (581, {'Department': 3, 'ManagementSalary': 0.0}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (65, {'Department': 4, 'ManagementSalary': nan})]

In [42]:
def salary_predictions():
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import accuracy_score
    import pandas as pd

    # Get Data into dataframe
    data = pd.DataFrame(index=G.nodes)
    data['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    data ['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    
    # Generate node features for classifier
    data['DegreeCentrality'] = data.index.to_series().map(nx.degree_centrality(G))
    data['Degree'] = data.index.to_series().map(nx.degree(G))
    data['BetweennessCentrality'] = data.index.to_series().map(nx.betweenness_centrality(G))
    data['PageRank'] = data.index.to_series().map(nx.pagerank(G))

    # print('Checkpoint 1:\n')
    # print(f'Data overview:\n {data.head(10)}')
    # print(f'NAN values in DF?:\n {data.isnull().any().any()}')
    # print('---------------\n\n\n')


    
    # featureset
    
    features = ['Department', 'DegreeCentrality', 'Degree', 'BetweennessCentrality','PageRank']
    target = ['ManagementSalary']

    # Train test split
    X_to_classify = data[data[target[0]].isna()]
    X = data[~data[target[0]].isna()]
    
    # print('Checkpoint 2:\n\n')
    # print(f'To Classify:\n {X_to_classify.head(10)}\n\n')
    # print(f'To Train:\n {X.head(10)}\n\n')
    # print(f'NAN values in To Classify?: {X.isnull().any().any()}\n')
    # print(f'NAN values in To Train?: {X.isnull().any().any()}\n')
    # print('---------------\n\n\n')
    
    y = X[target]
    X = X[features]
    
    
    
    # print(X, y)
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    
    # Score testing
    y_pred = clf.predict(X_test)
    
    # print(f'ROC_AUC score on training data: {roc_auc_score(y_test, y_pred)}')
    # print(f'Accuracy score on training data: {accuracy_score(y_test, y_pred)}')
    
    # Verification printout
    # print(f'to_predict length: {len(X_unknown)}')
    # print(f'train/test length: {len(X)}')
    # # print(X[:50])


    # Make final Prediction
    X_to_classify = X_to_classify[['Department', 'DegreeCentrality', 'Degree', 'BetweennessCentrality','PageRank']]
    X_transformed = scaler.transform(X_to_classify)
    X_predicted = clf.predict_proba(X_transformed)
    # raise NotImplementedError()
    
    
    X_predicted = pd.DataFrame(X_predicted, index=X_to_classify.index)
    answer = X_predicted[1]
    # print(type(answer))
    
    return answer


# salary_predictions()

1      0.04
65     0.97
18     0.07
215    0.75
283    1.00
       ... 
691    0.01
788    0.15
944    0.00
798    0.00
808    0.01
Name: 1, Length: 252, dtype: float64

In [ ]:
ans_salary_preds = salary_predictions()
assert type(ans_salary_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_salary_preds) == 252, "The series must be of length 252"


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [5]:
future_connections = pd.read_csv('assets/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to:      
1. Create a matrix of features of your choice for the edges found in `future_connections` using Networkx     
2. Train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data     
3. Predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [35]:
def new_connections_predictions():
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import accuracy_score
    import pandas as pd
    
    
    # Need edge data
    X = future_connections    
    X['edges'] = X.index
    

    #######################
    # Generating Features #
    #######################
    # print('Generating features...')

    # Jaccard Distance
    # print('...Jaccard Coefficient...')
    
    l = nx.jaccard_coefficient(G)
    j_dict = {}
    for u, v, p in l:
        j_dict[(u, v)] = p
    
    X['jaccard_cf'] = X['edges'].map(j_dict)
    

    # Resource Allocation
    # print('...Resource Allocation...')
    
    m = nx.resource_allocation_index(G)
    ra_dict = {}
    for u, v, p in m:
        ra_dict[(u, v)] = p
    
    X['resource_allocation'] = X['edges'].map(ra_dict)
    
    
    # Preferential Attachment
    # print('...Preferential Attachment...')
    
    n = nx.preferential_attachment(G)
    pa_dict = {}
    for u, v, p in n:
        pa_dict[(u, v)] = p
    
    X['preferential_attachmnent'] = X['edges'].map(pa_dict)
    
    # print('Done generating features!\n\n')
    

    ###################
    # Preprocess Data #
    ###################
    
    # print('Preprocessing Data...')
    
    # Separate to_predict from training
    # print('...Separating Data...')
    
    X_to_predict = X[X['Future Connection'].isna()]
    X = X[~X['Future Connection'].isna()]
    tuple_index = X.index

    y = X['Future Connection']
    X = X[['jaccard_cf', 'resource_allocation', 'preferential_attachmnent']]
    
    # print('...Scaling...')
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    # print('Done preprocessing!\n\n')

    
    ########################
    # Train the Classifier #
    ######################## 
    
    # print('Training the Classifier...')
    
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    

    
    # Verification
    # y_pred = clf.predict(X_test)
    # print(f'Testing Data Scores:')
    # print(f'AUC score: {roc_auc_score(y_pred, y_test)}')
    # print(f'Accuracy: {accuracy_score(y_pred, y_test)}\n\m')

    
    # print(f'Predicting final result')
    X_to_predict = X_to_predict[['jaccard_cf', 'resource_allocation', 'preferential_attachmnent']]
    X_transformed = scaler.transform(X_to_predict)
    y_predictions = clf.predict_proba(X_transformed)
    
    # raise NotImplementedError
    
    # X_to_predict = X.isna()
    # X = X.dropna()
    
    y_predictions = pd.DataFrame(y_predictions, index=X_to_predict.index)

    return y_predictions[1]

# new_connections_predictions()

(107, 348)    0.000000
(542, 751)    0.015518
(20, 426)     0.680000
(50, 989)     0.014534
(942, 986)    0.012492
                ...   
(165, 923)    0.016338
(673, 755)    0.007530
(939, 940)    0.012492
(555, 905)    0.018648
(75, 101)     0.000000
Name: 1, Length: 122112, dtype: float64

In [ ]:
ans_prob_preds = new_connections_predictions()
assert type(ans_prob_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_prob_preds) == 122112, "The series must be of length 122112"
